In [1]:
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [5]:
files_path = 'CleanFiles/'
files_format = '.csv'
files_postfix = 'Extract'
file_names = ['AgiaParaskevi','Aristotelous','Athens','Elefsina','Koropi','Liosia',
              'Lykovrisi','Marousi','NeaSmirni','Patision','Peristeri',
              'Pireus','Thrakomakedones']

In [6]:
def pm10_index(val):
    if val <= 25.0:
        return 0
    elif 26.0 <= val <= 50.0:
        return 1
    elif 51.0 <= val <= 90.0:
        return 2
    elif 91.0 <= val <= 180.0:
        return 3
    else:
        return 4

In [8]:
#neasmirni = pd.read_csv('CleanFiles/NeaSmirniExtract.csv', sep=',')
#neasmirni = neasmirni[['date_time','station_id','season','real_temp','real_windspd','real_winddir','real_humidity','forecast_tempC','forecast_windSpeed','forecast_windDirection','forecast_humidity','pm10']]
#neasmirni['pm10'] = neasmirni['pm10'].map(lambda a: pm10_index(a))
#temp = pd.read_csv('CleanFiles/PeristeriExtract.csv', sep=',')

In [4]:
T = 72
D = 1
X = []
Y = []
for t in range(len(neasmirni) - T):
    x = neasmirni[t:t+T]
    X.append(x)
    y = neasmirni[t:t+T]
    Y.append(y)